# Confidence intervals

Throughout the following exercises, you will learn to use Python to construct a confidence interval for a point estimate. Before starting on this programming exercise, we strongly recommend watching the video lecture and completing the IVQ for the associated topics.



All the information you need for solving this assignment is in this notebook, and all the code you will be implementing will take place within this notebook. 

As we move forward, you can find instructions on how to install required libraries as they arise in this notebook. Before we begin with the exercises and analyzing the data, we need to import all libraries and extensions required for this programming exercise. Throughout the course, we will be using numpy, pandas, and scipy stats for operations.

In [1]:
# Import packages
# Import the relevant Python libraries and modules needed in this lab.

### YOUR CODE HERE ###
# Import libraries and packages

# Import packages
from google.cloud import storage

import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from matplotlib import cm
from datetime import datetime
import glob
import os
from io import StringIO
from io import BytesIO
import json
import pickle
import six
import charset_normalizer
from wordcloud import WordCloud 
from typing import List

from scipy import stats
import statsmodels.api as sm

sns.set()
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None

In [2]:
#storage_client =  storage.Client.from_service_account_json('../heidless-jupyter-0-d2008100d98c.json')
storage_client =  storage.Client()

BUCKET_NAME = 'heidless-jupyter-bucket-0'

bucket = storage_client.get_bucket(BUCKET_NAME)

AllCSV = []
my_prefix = 'education/'
my_file = 'education_districtwise.csv'
full_file = my_prefix + my_file
#print(f'full_file: {full_file}')

file_names = list(bucket.list_blobs(prefix=my_prefix))
for file in file_names:
    if(file.name != my_prefix):
        if file.name == full_file:
            AllCSV.append(file.name)
#            print(file.name)
AllCSV


['education/education_districtwise.csv']

In [3]:
# RUN THIS CELL TO IMPORT YOUR DATA.

### YOUR CODE HERE ###
#companies = pd.read_csv("Unicorn_Companies.csv")

# Display the first 10 rows of the data
all_dataframes = []

for csv in AllCSV:
    blob = bucket.get_blob(csv)
    if blob is not None and blob.exists(storage_client):
        bt = blob.download_as_string()
        s = str(bt, 'ISO-8859-1')
        s = StringIO(s)
        df = pd.read_csv(s, encoding='ISO-8859-1', low_memory=False)

        #df['country'] = csv[0:2] # adding column 'country' so that each dataset could be identified uniquely
        all_dataframes.append(df)
        print(csv)
    
#all_dataframes[0].head() # index 0 to 9 for [CA, DE, FR, GB, IN, JP, KR, MX, RU, US] datasets

education_districtwise = all_dataframes[0]
education_districtwise.head()


education/education_districtwise.csv


,DISTNAME,STATNAME,BLOCKS,VILLAGES,CLUSTERS,TOTPOPULAT,OVERALL_LI
0,DISTRICT32,STATE1,13,391,104,875564.0,66.92
1,DISTRICT649,STATE1,18,678,144,1015503.0,66.93
2,DISTRICT229,STATE1,8,94,65,1269751.0,71.21
3,DISTRICT259,STATE1,13,523,104,735753.0,57.98
4,DISTRICT486,STATE1,8,359,64,570060.0,65.00


import numpy as np
import pandas as pd
from scipy import stats

education_districtwise = pd.read_csv("education_districtwise.csv")
education_districtwise = education_districtwise.dropna()

We’ll continue with our previous scenario, in which you’re a data professional working for the Department of Education of a large nation. Earlier, we imagined that the Department of Education asked you to collect the data on district literacy rates. You were only able to survey 50 randomly chosen districts, instead of all 634 districts included in your original dataset. You used Python to simulate taking a random sample of 50 districts, and make a point estimate of the population mean, or literacy rate for *all* districts. 

Now imagine that the department asks you to construct a 95% confidence interval for your estimate of the mean district literacy rate. You can use Python to construct the confidence interval. 


You can also use the same sample data that you worked with earlier. Write the code to have Python simulate the same random sample of district literacy rate data. First, name your variable `sampled_data`. Then, enter the arguments of the `sample()` function. 


*   `n`: Your sample size is `50`. 
*   `replace`: Choose `True` because you are sampling with replacement.
*   `random_state`: Choose the same random number to generate the same results–previously, you used `31,208`. 






 


In [4]:
sampled_data = education_districtwise.sample(n=50, replace=True, random_state=31208)
sampled_data

,DISTNAME,STATNAME,BLOCKS,VILLAGES,CLUSTERS,TOTPOPULAT,OVERALL_LI
615,DISTRICT606,STATE20,6,782,96,NaN,NaN
210,DISTRICT618,STATE28,5,183,34,656916.0,56.00
348,DISTRICT255,STATE34,10,920,78,1670931.0,73.86
248,DISTRICT63,STATE18,8,230,32,146742.0,82.06
276,DISTRICT57,STATE27,6,121,6,270063.0,85.58
350,DISTRICT254,STATE34,33,3142,286,7102430.0,67.53
252,DISTRICT194,STATE3,8,271,24,78413.0,62.48
10,DISTRICT412,STATE1,11,187,95,476820.0,68.69
478,DISTRICT155,STATE22,5,1116,58,1311008.0,71.11
638,DISTRICT609,STATE6,18,1470,167,2468965.0,74.72


The output shows 50 districts selected randomly from your dataset. Each has a different literacy rate. 

## Construct a 95% confidence interval 

Now, construct a 95% confidence interval of the mean district literacy rate based on your sample data. Recall the four steps for constructing a confidence interval:

1.   Identify a sample statistic
2.   Choose a confidence level
3.   Find the margin of error 
4.   Calculate the interval

### `scipy.stats.norm.interval()`

Earlier, you worked through these steps one by one to construct a confidence interval. With Python, you can construct a confidence interval with just a single line of code–and get your results faster! 

If you’re working with a large sample size, say larger than 30, you can construct a confidence interval for the mean using `scipy.stats.norm.interval()`. This function includes the following arguments: 

*   `alpha`: The confidence level
*   `loc`: The sample mean
*   `scale`: The sample standard error

**Reference**: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html.

Let’s explore each argument in more detail. 



#### `alpha`: The confidence level

The Department of Education requests a confidence level of 95%, which is the accepted standard for government funded research. 

#### `loc`: The sample mean

This is the mean literacy rate of your sample of 50 districts. Name a new variable `sample_mean`. Then, compute the mean district literacy rate for your sample data. 

In [5]:
sample_mean = sampled_data['OVERALL_LI'].mean()

#### `scale`: The sample standard error

Recall that **standard error** measures the variability of your sample data. You may remember that the formula for the sample standard error is the sample standard deviation divided by the square root of the sample size.

**Note**: In practice, we typically don't know the true standard error, so we replace it with the estimated standard error.

You can write code to express the formula and have Python do the calculation for you: 

1. Name a new variable `estimated_standard_error`. 
2. Take the standard deviation of your sample data, and divide by the square root of your sample. 
3. In parentheses, write the name of your data frame followed by the shape function and zero in brackets. Recall that the shape function returns the number of rows and columns in a data frame. `shape[0]` returns only the number of rows, which is the same number as your sample size.  

In [6]:
estimated_standard_error = sampled_data['OVERALL_LI'].std() / np.sqrt(sampled_data.shape[0])

Now you’re ready to put all this together to construct your confidence interval for the mean using `scipy.stats.norm.interval()`. First, write out the function and set the arguments:

*   `alpha`: Enter `0.95` because you want to use a 95% confidence level
*   `loc`: Enter the variable `sample_mean`
*   `scale`: Enter the variable `estimated_standard_error`


In [8]:
stats.norm.interval(confidence=0.95, loc=sample_mean, scale=estimated_standard_error)

(71.73192867194474, 77.02424154082122)

You have a 95% confidence interval for the mean district literacy rate that stretches from about 71.4% to 77.0%. 

95% CI: (71.42, 77.02)

The Department of Education will use your estimate of the mean district literacy rate to help make decisions about distributing funds to different states.  

## Construct a 99% confidence interval 

Now imagine that a senior director in the department wants to be even *more* confident about your results. The director wants to make sure you have a reliable estimate, and suggests that you recalculate your interval with a 99% confidence level.

To compute a 99% confidence interval based on the same sample data, just change `alpha` to `0.99`. 



In [9]:
stats.norm.interval(confidence=0.99, loc=sample_mean, scale=estimated_standard_error)

(70.90044605823485, 77.85572415453112)

You have a 99% confidence interval for the mean district literacy rate that stretches from about 70.5% to 77.9%.

99% CI: (70.54, 77.90)

### Relationship between confidence level and confidence interval

You may notice that as the confidence *level* gets higher, the confidence *interval* gets wider. 

* With a confidence level of 95%, the interval covers 5.6 percentage points (71.4% - 77.0%)
* With a confidence level of 99%, the interval covers 7.4 percentage points (70.5% - 77.9%)

This is because a wider confidence interval is more likely to include the actual population parameter.

Your results will help the Department of Education decide how to distribute government resources to improve literacy. 

If you have successfully completed the material above, congratulations! You now understand how to use Python to construct a confidence interval for a point estimate. Going forward, you can start using Python to construct confidence intervals for your own data.